## Crafting an Interactive Narrative with MiniAutoGen
This example demonstrates how MiniAutoGen can be utilized to create an interactive, multi-character narrative set in a Parisian café. It showcases the integration of advanced language models, custom templates, and dynamic agent interactions to build an engaging story.

### Setting Up the Environment
First, ensure the MiniAutoGen library and dependencies are installed. Use the dotenv library for environment variable management.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# Initializing LLM Clients
## Create clients for interacting with OpenAI's models and LiteLLM.

from miniautogen.llms.llm_client import LiteLLMClient

openai_client = LiteLLMClient(model='gpt-3.5-turbo-16k')

In [3]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components.components import (
    Jinja2TemplatesComponent, LLMResponseComponent, UserResponseComponent, 
    AgentReplyComponent, TerminateChatComponent, NextAgentSelectorComponent
)

# Create Jinja2 template components for narrative and conversation history
PROMPT_TEMPLATE_MOVIE_SCENE = """
# Introduction
- You are a character in a movie, as described in the "Your Character" section.
- You interact in a movie setting, collaborating with other characters to develop a specific scene.

# Movie Scene
- Scene description: {{chat.context['goal']}}

# Your Character
- CHARACTER NAME: {{agent.name}}
- CHARACTER DESCRIPTION: 
{{agent.role}}

# Scene Characters
{% for agent in chat.agentList %}
  - {{agent.name}}: {{agent.role}}
{% endfor %}

# Scene Dynamics
- Maintain consistency with the plot and personality of your character.
- Your identity: You are {{agent.name}}, keep a clear distinction between yourself and other characters.
- Speaker identification: Each line will be preceded by the character's name.

# Scene Instructions
- Stay true to the scene's objective and tone.
- Contribute to the development of the narrative and interactions between characters.

# Response Format
- Respond ONLY with your character's lines or actions.
- Ensure that your contribution advances the narrative or deepens the understanding of the characters and their relationships.
"""


PROMPT_TEMPLATE_CONVERSATION_HISTORY = """
CONVERSATION HISTORY:
-----------------------------------------------------------------------------------------
[
  {% for message in messages %}
    {"Sender": "{{ message['sender_id'] }}", "Content": "{{ message['message'] | escape }}"}{% if not loop.last %}, {% endif %}
  {% endfor %}
]
-----------------------------------------------------------------------------------------
"""

jinja_component = Jinja2TemplatesComponent()
jinja_component.add_template(PROMPT_TEMPLATE_MOVIE_SCENE, 'system')
jinja_component.add_template(PROMPT_TEMPLATE_CONVERSATION_HISTORY, 'user')


chat_context = {'goal': '''
Title: "Meeting at Café Parisiense"
In a charming Parisian café, three characters cross paths on a sunny spring afternoon. The café, adorned with outdoor tables, is filled with flowers in full bloom, while the gentle notes of a violin enchant the patrons.
Alex, a talented local artist, frequents the café to find inspiration in every stroke of his imagination. Jordan, an adventurous American tourist, is experiencing the beauty of Paris for the first time. Camille, the captivating café waitress, is known for both her radiant kindness and captivating beauty. What neither of the two strangers knows is that they are secretly in love with Camille.
As the three come together in a scene filled with captivating dialogues, stolen glances, and an atmosphere charged with budding romance, each tries to win Camille's heart in their own way. Alex seeks to express his love through his drawings, while Jordan dazzles Camille with his charm and stories of his travels around the world. Camille, in turn, finds herself torn between the affection she feels for the local artist and the curiosity awakened by the mysterious stranger in her heart.
As the scene unfolds, comedic misunderstandings and moments of tenderness arise, challenging expectations and fueling the romantic tension that hangs in the air. With every gesture, every whispered word, and every exchanged glance, the enigma of the destiny of these three characters unravels in the charming setting of Café Parisiense, where love blossoms under the light of the City of Lights.
'''}

# Initialize the chat environment and set the context
chat = Chat()
chat.context = chat_context

# Create the pipelines for the different agents
pipeline_jinja = Pipeline([jinja_component, LLMResponseComponent(openai_client)])
pipeline_admin = Pipeline(
    [NextAgentSelectorComponent(), AgentReplyComponent(), TerminateChatComponent()])




In [4]:
FIRST_MESSAGE =  """
On a spring afternoon in Paris, the sun poured its golden light over the cobblestone streets of the city, illuminating the scene like an Impressionist painting coming to life. At the heart of the French capital lay Café Parisiense, a picturesque refuge that seemed to have sprung from a 19th-century romance.

The café's tables sprawled beneath the canopy of flowering trees, their green leaves swaying gently to the rhythm of a delicate breeze. Each table was adorned with fresh bouquets of colorful flowers, and the soft sound of a violin in the background filled the air with melodic notes, as if Paris were composing its own romantic soundtrack.

In this idyllic setting, three destinies intertwined on that day. The first character, Alex, an exceptionally talented local artist, sat at one of the café's tables, his gaze fixed on the sketchbook before him. He frequented Café Parisiense regularly, drawing inspiration from urban landscapes and occasionally from the people who gathered there. However, on that day, his attention was focused on a particular figure: Camille, the gracious café waitress known for her kindness and beauty.

While Alex dedicated himself to capturing Camille's essence in charcoal strokes, another character, Jordan, entered the café. Jordan was an adventurous American tourist, and Paris was the first stop on his European journey. His curious eyes met the charming ambiance of Café Parisiense, and he was drawn to the aroma of fresh coffee and the romantic atmosphere that permeated the place.

And so, the stage was set for a meeting that would change the course of their lives. Alex and Jordan, two strangers unaware of each other's existence, were about to cross paths at Café Parisiense, each secretly enamored with Camille. As the violin continued to play love melodies in the background, destiny began to weave its web of intrigue and passions, casting the first rays of what would become an unforgettable romance.
"""

json_messages = [
    {'sender_id': 'storyteller', 'message': FIRST_MESSAGE},
]

chat.add_messages(json_messages)

In [5]:
# Create and configure agents

ALEX = """
Full Name: Alex Dubois
Age: 30 years old
Physical Description: Tall and slender, with tousled brown hair and expressive green eyes. He always wears bohemian clothing and has an enigmatic gaze.
Personality: Artistic and introspective, Alex is a talented painter who finds inspiration in every corner of Paris. He is kind, sensitive, and passionate about his art, often getting lost in his creative thoughts.
Background: Alex grew up in Paris and studied arts at the renowned École des Beaux-Arts. He is known in the city for his art exhibitions but keeps a reserved personal life.
"""

JORDAN = """
Full Name: Jordan Anderson
Age: 28 years old
Physical Description: Athletic and of average height, with blond hair and blue eyes. He has a charismatic appearance and a captivating smile.
Personality: Outgoing, adventurous, and always in search of new experiences. Jordan is a natural storyteller with a free spirit. He travels the world in pursuit of adventures and beauty.
Background: Jordan was born in the United States and works as a travel journalist. His career has led him to explore various countries, but Paris is his most recent and exciting destination.
"""

CAMILLE = """
Full Name: Camille Moreau
Age: 25 years old
Physical Description: An elegant young woman with wavy brown hair that falls softly over her shoulders and bright green eyes. She possesses a natural charm.
Personality: Friendly, sweet, and attentive, Camille is known for her hospitality and warm smile. She loves Parisian culture and dreams of exploring the world someday.
Background: Camille grew up in Paris and has been working at Café Parisiense since she was a teenager. She is passionate about classical music and aspires to travel and study the arts someday.
"""


storyteller = """
The narrator of this story is a subtle and omniscient presence, guiding the reader through the intricacies of the romance unfolding at the Café Parisiense. With a gentle and engaging voice, the narrator inserts into the events with the grace of a keen observer, bringing forth the romantic and magical atmosphere of the City of Lights.
The narrative is like a richly painted canvas, detailing the colors, sounds, and emotions that permeate each scene. It invites us to dive into the psyche of the characters, revealing their deepest thoughts and feelings while simultaneously keeping us suspended in the enchantment of the city of Paris.
This narrator acts as a guide, leading us through the sun-drenched cobblestone streets and the tables adorned with fresh flowers at Café Parisiense. It invites us to contemplate the beauty of subtle gestures, stolen glances, and unspoken words that weave the fabric of love and desire among the characters.
With a touch of mystery and a healthy dose of romance, the narrator guides us as destiny unfolds at Café Parisiense, where passions intertwine, secrets are revealed, and love blossoms under the radiant light of Paris. Get ready to embark on this enchanting journey, where the pages come to life, and the heart beats in harmony with the sighs of the City of Lights.
"""

agente1_data = {
    'agent_id': 'ALEX',
    'name': 'Alex Dubois',
    'role': ALEX}

agente2_data = {
    'agent_id': 'JORDAN',
    'name': 'Jordan Anderson',
    'role': JORDAN}

agente3_data = {
    'agent_id': 'CAMILLE',
    'name': 'Camille Moreau',
    'role': 'CAMILLE'}

agente4_data = {
    'agent_id': 'storyteller',
    'name': 'storyteller',
    'role': 'storyteller'}

# Criação de Agentes
agent1 = Agent.from_json(agente1_data)
agent1.pipeline = pipeline_jinja  # Atribuindo o pipeline ao agente

agent2 = Agent.from_json(agente2_data)
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente

agent3 = Agent.from_json(agente3_data)
agent3.pipeline = pipeline_jinja  # Atribuindo o pipeline_user ao terceiro agente

agent4 = Agent.from_json(agente4_data)
agent4.pipeline = pipeline_jinja  # Atribuindo o pipeline_user ao terceiro agente

# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)
chat.add_agent(agent3)
chat.add_agent(agent4)

# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, 5)

In [6]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1


[{'role': 'system', 'content': '\n# Introduction\n- You are a character in a movie, as described in the "Your Character" section.\n- You interact in a movie setting, collaborating with other characters to develop a specific scene.\n\n# Movie Scene\n- Scene description: \nTitle: &#34;Meeting at Café Parisiense&#34;\nIn a charming Parisian café, three characters cross paths on a sunny spring afternoon. The café, adorned with outdoor tables, is filled with flowers in full bloom, while the gentle notes of a violin enchant the patrons.\nAlex, a talented local artist, frequents the café to find inspiration in every stroke of his imagination. Jordan, an adventurous American tourist, is experiencing the beauty of Paris for the first time. Camille, the captivating café waitress, is known for both her radiant kindness and captivating beauty. What neither of the two strangers knows is that they are secretly in love with Camille.\nAs the three come together in a scene filled with captivating dialo

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 2


Alex: *sitting at the table, lost in his sketchbook, capturing the essence of Camille with each stroke of charcoal*
[{'role': 'system', 'content': '\n# Introduction\n- You are a character in a movie, as described in the "Your Character" section.\n- You interact in a movie setting, collaborating with other characters to develop a specific scene.\n\n# Movie Scene\n- Scene description: \nTitle: &#34;Meeting at Café Parisiense&#34;\nIn a charming Parisian café, three characters cross paths on a sunny spring afternoon. The café, adorned with outdoor tables, is filled with flowers in full bloom, while the gentle notes of a violin enchant the patrons.\nAlex, a talented local artist, frequents the café to find inspiration in every stroke of his imagination. Jordan, an adventurous American tourist, is experiencing the beauty of Paris for the first time. Camille, the captivating café waitress, is known for both her radiant kindness and captivating beauty. What neither of the two strangers knows 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 3


...

CAMILLE: *glances over at Alex, noticing his focused gaze on his sketchbook* He always manages to capture the beauty of this place so effortlessly. *walks over to Alex's table with a warm smile* Bonjour, Alex. Your art continues to amaze me.
[{'role': 'system', 'content': '\n# Introduction\n- You are a character in a movie, as described in the "Your Character" section.\n- You interact in a movie setting, collaborating with other characters to develop a specific scene.\n\n# Movie Scene\n- Scene description: \nTitle: &#34;Meeting at Café Parisiense&#34;\nIn a charming Parisian café, three characters cross paths on a sunny spring afternoon. The café, adorned with outdoor tables, is filled with flowers in full bloom, while the gentle notes of a violin enchant the patrons.\nAlex, a talented local artist, frequents the café to find inspiration in every stroke of his imagination. Jordan, an adventurous American tourist, is experiencing the beauty of Paris for the first time. Camille, the capt

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Executing round 5


storyteller: As Jordan approached Alex's table, the two strangers exchanged a curious glance, unknowingly connected by their shared admiration for Camille. The air was charged with tension, as each man silently sized up the other, gauging their intentions.

Alex looked up from his sketchbook, his green eyes meeting Jordan's blue ones. He gave a polite smile and gestured to the empty seat across from him.

Alex: "Of course, please, have a seat. It's always nice to meet someone new in this enchanting café."

Jordan took the offered seat, his smile widening a little.

Jordan: "Thank you. My name's Jordan, by the way. I couldn't help but notice your skill with the charcoal. Are you an artist?"

Alex nodded, a spark of pride flickering in his eyes.

Alex: "Yes, I am. My name's Alex. I find inspiration in the everyday beauty that surrounds us, and I often come to Café Parisiense to capture its essence on paper."

Camille approached the table, noticing the newfound companionship.

Camille: "B

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


Alex: *smiles politely at Jordan* It's a pleasure to meet you, Jordan. I'm Alex. And yes, I am an artist. I find inspiration in the beauty of everyday life, especially here at Café Parisiense.

*glances at Camille, his eyes filled with warmth*

Camille, your presence always brings an extra touch of magic to this place. Please, join us.

*waits for Camille to sit down, his heart racing with anticipation*


In [7]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'storyteller',
  'message': "\nOn a spring afternoon in Paris, the sun poured its golden light over the cobblestone streets of the city, illuminating the scene like an Impressionist painting coming to life. At the heart of the French capital lay Café Parisiense, a picturesque refuge that seemed to have sprung from a 19th-century romance.\n\nThe café's tables sprawled beneath the canopy of flowering trees, their green leaves swaying gently to the rhythm of a delicate breeze. Each table was adorned with fresh bouquets of colorful flowers, and the soft sound of a violin in the background filled the air with melodic notes, as if Paris were composing its own romantic soundtrack.\n\nIn this idyllic setting, three destinies intertwined on that day. The first character, Alex, an exceptionally talented local artist, sat at one of the café's tables, his gaze fixed on the sketchbook before him. He frequented Café Parisiense regularly, drawing inspiration from urban lands

In [8]:
table_md = chat.get_messages()[['sender_id', 'message']]
# Especifique o caminho do arquivo onde deseja salvar o arquivo Markdown
file_path = 'chat_movie.md'

# Abra o arquivo para escrever e salve os registros no formato "Sender_id\nMessage"
with open(file_path, 'w') as file:
    for index, row in table_md.iterrows():
        sender_id = row['sender_id']
        message = row['message']
        
        # Adicionar um cabeçalho com o sender_id em negrito
        file.write(f'### **Sender_id:** {sender_id}\n\n')
        
        # Adicionar uma linha divisória criativa
        file.write('***\n\n')
        
        # Adicionar o conteúdo da mensagem
        file.write(message)
        file.write('\n\n')
        file.write('\n\n')
        file.write('\n\n')